In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import subprocess

try:
    import japanize_matplotlib
    print("japanize_matplotlib is already installed.")
except ImportError:
    !pip install japanize_matplotlib
    print("japanize_matplotlib has been installed.")

try:
    import catboost
    print("catboost is already installed.")
except ImportError:
    !pip install catboost
    print("catboost has been installed.")

try:
    import optuna
    print("optuna is already installed.")
except ImportError:
    !pip install optuna
    print("optuna has been installed.")

try:
    import geolib
    print("geolib is already installed.")
except ImportError:
    !pip install geolib
    print("geolib has been installed.")

# mkdirとechoコマンドの実行
try:
    subprocess.check_call('mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd', shell=True)
    print('Directory and file for OpenCL vendors have been created.')
except subprocess.CalledProcessError as e:
    print(f"Error running mkdir and echo command: {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 65.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=db25fe676b7fc1feea3bc3b55df4886bc9daada899ee44f1e35893e2fcc35633
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib
japanize_matplotlib has been installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 20.6 MB/s eta 0:00:00
catboost has been installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00
optuna has been installed.
geolib has been installed.
Directory and file for OpenCL vendors have been created.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import japanize_matplotlib
import seaborn as sns
import pickle

import sys
sys.path.append('/content/drive/MyDrive/yuto/project_directory/code')

from utils import (
                   to_cat_col,
                   VotingModel,
                   check_dataframe_elements
                   )

TOTAL_PRICE_LOG="取引価格（総額）_log"
PRICE_PER_AREA_LOG="単位面積あたりの取引価格_log"

model_cat_lq_cv_path="/content/drive/MyDrive/yuto/project_directory/models/model_cat_lq_cv_20240908_141325_0_06609571831620502.pkl"
model_lgbm_l1_cv_path="/content/drive/MyDrive/yuto/project_directory/models/model_lgbm_l1_cv_20240908_110735_0_06585289841.pkl"
#model_lgbm_l1_cv_path="/content/drive/MyDrive/yuto/project_directory/models/model_lgbm_l1_cv_sub_sub_0_06585289841760715.pkl"
model_lgbm_l1_cv_target_path="/content/drive/MyDrive/yuto/project_directory/models/model_lgbm_l1_cv_target_20240910_144521_0_06636916360273092.pkl"

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
def visualize_hyperparameters(estimators):
    if isinstance(estimators, VotingModel):  # 入力がVotingModelかどうかを確認します
        for estimator in estimators.estimators:  # VotingModel内の推定器を反復処理します
            params = estimator.get_params()
            print(f"Estimator: {estimator.__class__.__name__}")  # 推定器のタイプを表示します
            #for param, value in params.items():
            #    print(f"  {param}: {value}")
            print(params)
    else:
        params = estimators.get_params()
        print(f"Estimator: {estimators.__class__.__name__}")  # 推定器のタイプを表示します
        #for param, value in params.items():
        #    print(f"  {param}: {value}")
        print(params)

# モデルの読み込み
def read_model(model_path):
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    return model

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/preprocess_main_table.csv",index_col=0)
df=df[['dataset', 'ID', '価格情報区分', '都道府県名', '市区町村名', '地区名', '住所不一致_Flag', '住所',
       '最寄駅：名称', '最寄駅：距離（分）', '徒歩時間_10分単位', 'x_api', 'y_api', 'lon', 'lat',
       '間取り', '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '単位面積あたりの取引価格_log', '建築西暦年', '築年数',
       'log_面積（㎡）', '購入までの築年数', '面積（㎡）-最寄駅：距離（分）', '面積（㎡）-築年数',
       '面積（㎡）容積率（％）_combi', '旧耐震フラグ', '最寄駅までの距離', 'geohash_2', 'geohash_3',
       'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7', '県_市_count',
       '建築年_count', '県_市_geohash_2_駅数', '県_市_geohash_3_駅数', '県_市_geohash_4_駅数',
       '県_市_geohash_5_駅数', '県_市_geohash_6_駅数', '県_市_geohash_7_駅数',
       '県_市_geohash_2_駅徒歩時間平均', '県_市_geohash_2_建築年平均', '県_市_geohash_3_駅徒歩時間平均',
       '県_市_geohash_3_建築年平均', '県_市_geohash_4_駅徒歩時間平均', '県_市_geohash_4_建築年平均',
       '県_市_geohash_5_駅徒歩時間平均', '県_市_geohash_5_建築年平均', '県_市_geohash_6_駅徒歩時間平均',
       '県_市_geohash_6_建築年平均', '県_市_geohash_7_駅徒歩時間平均', '県_市_geohash_7_建築年平均']]
#df=df.drop(columns=["住所_地区名欠損フラグ","address","title",TOTAL_PRICE_LOG],axis=1)

In [ ]:
# データフレームを分割
df_train = df[df['dataset'] == 'train'].set_index('ID')
df_test = df[df['dataset'] == 'test'].set_index('ID')

# datasetカラムを削除
df_train = df_train.drop(columns=['dataset'])
df_test = df_test.drop(columns=['dataset'])

df_train,cat_cols=to_cat_col(df_train)

#print("Train DataFrame:")
#print(df_train)
#print("\nTest DataFrame:")
#print(df_test)

In [ ]:
#check_dataframe_elements(df)

In [ ]:
def decode_price(y_pred,df_test):
    # 単位面積あたりの取引価格の対数から10のべき乗を取る
    price_per_area = 10**y_pred_lgbm
    # その値に面積を掛けて、取引価格（総額）を計算
    predicted_price_total = price_per_area * df_test["面積（㎡）"]
    # 取引価格（総額）の対数を計算
    y_pred= np.log10(predicted_price_total)
    #y_pred=y_pred.rename(columns={"面積（㎡）":"取引価格（総額）_log"})
    y_pred.name = "取引価格（総額）_log"
    return y_pred

In [ ]:
#print(abc)

Catboost l1

In [ ]:
# 損失関数lqでトレーニングしたCatboostモデルを呼び出す
#model_cat_lq_cv=read_model(model_cat_lq_cv_path)
#visualize_hyperparameters(model_cat_lq_cv)

#y_pred_cat_lq_cv=pd.Series(model_cat_lq_cv.predict_cat(df_test.drop(TOTAL_PRICE_LOG,axis=1)),index=df_test.index)
#y_pred_cat=pd.Series(model_cat_lq_cv.predict_cat(df_test.drop(TOTAL_PRICE_LOG,axis=1)),index=df_test.index)
#y_pred_cat=decode_price(y_pred_cat,df_test)
# Seriesの名前を変更
#y_pred_cat.name = "取引価格（総額）_log"

# y_predをデータフレームに変換
#df_pred = pd.DataFrame(y_pred_cat, columns=["取引価格（総額）_log"])

# 結果を表示
#print(df_pred)

#df_pred.to_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/one_model_predictions/test_cat_cv.csv")

In [ ]:
#del model_cat_lq_cv,df_pred

LightGBM l1 CV

In [ ]:
model_lghm_l1_cv=read_model(model_lgbm_l1_cv_path)

In [ ]:
visualize_hyperparameters(model_lghm_l1_cv)

#y_pred_lgbm_l1_cv=pd.Series(model_lghm_l1_cv.predict_lgbm(df_test.drop(PRICE_PER_AREA_LOG,axis=1)),index=df_test.index)
y_pred_lgbm=pd.Series(model_lghm_l1_cv.predict_lgbm(df_test.drop(PRICE_PER_AREA_LOG,axis=1)),index=df_test.index)
y_pred_lgbm=decode_price(y_pred_lgbm,df_test)
# Seriesの名前を変更
y_pred_lgbm.name = "取引価格（総額）_log"

# y_predをデータフレームに変換
df_pred = pd.DataFrame(y_pred_lgbm, columns=["取引価格（総額）_log"])

# 結果を表示
print(df_pred)

df_pred.to_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/one_model_predictions/test_lgbm_cv.csv")

Estimator: LGBMRegressor
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.2661179203230455, 'importance_type': 'split', 'learning_rate': 0.04804216261746864, 'max_depth': 128, 'min_child_samples': 577, 'min_child_weight': 5.774925183375773, 'min_split_gain': 0.3914873364805788, 'n_estimators': 46022, 'n_jobs': None, 'num_leaves': 31, 'objective': 'l1', 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 0.528846990441364, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'metric': 'mae', 'colsample_bynode': 0.216985581379894, 'bagging_fraction': 0.9760006064193019, 'feature_fraction': 0.5336978769826026, 'max_bin': 1123, 'epsilon': 4.954717758504798, 'alpha': 0.5725841971738328, 'min_sum_hessian_in_leaf': 1.9378627484627984, 'boost_from_average': False, 'verbose': -1}
Estimator: LGBMRegressor
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.2661179203230455, 'importance_type': 'split', 'learning_rate': 0.04804216261746864, 'ma

In [ ]:
df_pred_tg=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/one_model_predictions/test_lgbm_cv20240908_111634.csv")
print(df_pred_tg)

             ID  取引価格（総額）_log
0       1000107      6.754242
1       1000109      7.312165
2       1000112      6.674249
3       1000114      6.474699
4       1000117      7.387694
...         ...           ...
54397  47036620      7.519506
54398  47037115      7.450802
54399  47037850      7.241828
54400  47037855      7.523497
54401  47037856      7.563230

[54402 rows x 2 columns]


In [ ]:
#del model_lghm_l1_cv,df_pred

LightGBM l1 CV target